In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pwd
%cd ./drive/My Drive/Colab Notebooks/UniverseDacon/
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
sample_submission = pd.read_csv('sample_submission.csv', index_col=0)
sample_submission.describe()

/content
/content/drive/My Drive/Colab Notebooks/UniverseDacon


,STAR_WHITE_DWARF,STAR_CATY_VAR,STAR_BROWN_DWARF,SERENDIPITY_RED,REDDEN_STD,STAR_BHB,GALAXY,SERENDIPITY_DISTANT,QSO,SKY,STAR_RED_DWARF,ROSAT_D,STAR_PN,SERENDIPITY_FIRST,STAR_CARBON,SPECTROPHOTO_STD,STAR_SUB_DWARF,SERENDIPITY_MANUAL,SERENDIPITY_BLUE
count,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
# TRAIN 데이터의 TYPE을 SAMPLE_SUBMISSION에 대응하는 가변수 형태로 변환시킵니다.
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

In [4]:
from collections import Counter
print(sorted(Counter(train['type_num']).items()))
type0 = train[train['type_num']==1]
type1 = train[train['type_num']==4]
type2 = train[train['type_num']==5]
type3 = train[train['type_num']==6]
type4 = train[train['type_num']==8]
type5 = train[train['type_num']==10]
type6 = train[train['type_num']==11]
type7 = train[train['type_num']==13]
type8 = train[train['type_num']==15]
type9 = train[train['type_num']==18]
type10 = train[train['type_num']==0]
type11 = train[train['type_num']==2]
type12 = train[train['type_num']==3]
type13 = train[train['type_num']==7]
type14 = train[train['type_num']==9]
type15 = train[train['type_num']==12]
type16 = train[train['type_num']==14]
type17 = train[train['type_num']==16]
type18 = train[train['type_num']==17]
unsam_train0 = type0.sample(5000)
unsam_train1 = type1.sample(5000)
unsam_train2 = type2.sample(5000)
unsam_train3 = type3.sample(5000)
unsam_train4 = type4.sample(5000)
unsam_train5 = type5.sample(5000)
unsam_train6 = type6.sample(5000)
unsam_train7 = type7.sample(5000)
unsam_train8 = type8.sample(5000)
unsam_train9 = type9.sample(5000)
unsam_train = pd.concat([unsam_train0, unsam_train1, unsam_train2, unsam_train3, unsam_train4, unsam_train5, unsam_train6, unsam_train7, unsam_train8, unsam_train9, type10, type11, type12, type13, type14, type15, type16, type17, type18],axis=0)
print(sorted(Counter(unsam_train['type_num']).items()))


[(0, 2160), (1, 6506), (2, 500), (3, 2562), (4, 14618), (5, 13500), (6, 37347), (7, 4654), (8, 49680), (9, 127), (10, 13750), (11, 6580), (12, 13), (13, 7132), (14, 3257), (15, 14630), (16, 1154), (17, 61), (18, 21760)]
[(0, 2160), (1, 5000), (2, 500), (3, 2562), (4, 5000), (5, 5000), (6, 5000), (7, 4654), (8, 5000), (9, 127), (10, 5000), (11, 5000), (12, 13), (13, 5000), (14, 3257), (15, 5000), (16, 1154), (17, 61), (18, 5000)]


train 에서 type column 을 없애고 test 랑 합쳐서 .

IQR 방법으로 Outlier 제거.
일단 주석처리.

In [5]:
y_train = unsam_train['type_num']
train_wo_type = unsam_train.drop(columns=['type','type_num','fiberID'], axis=1)
test = test.drop(columns=['fiberID'])
print(len(train_wo_type))
total = pd.concat([train_wo_type, test])
total.info()

64488
<class 'pandas.core.frame.DataFrame'>
Int64Index: 74497 entries, 90216 to 10008
Data columns (total 21 columns):
id            74497 non-null int64
psfMag_u      74497 non-null float64
psfMag_g      74497 non-null float64
psfMag_r      74497 non-null float64
psfMag_i      74497 non-null float64
psfMag_z      74497 non-null float64
fiberMag_u    74497 non-null float64
fiberMag_g    74497 non-null float64
fiberMag_r    74497 non-null float64
fiberMag_i    74497 non-null float64
fiberMag_z    74497 non-null float64
petroMag_u    74497 non-null float64
petroMag_g    74497 non-null float64
petroMag_r    74497 non-null float64
petroMag_i    74497 non-null float64
petroMag_z    74497 non-null float64
modelMag_u    74497 non-null float64
modelMag_g    74497 non-null float64
modelMag_r    74497 non-null float64
modelMag_i    74497 non-null float64
modelMag_z    74497 non-null float64
dtypes: float64(20), int64(1)
memory usage: 12.5 MB


In [0]:
# from collections import Counter
# def detect_outliers(df, n, features): 
#   outlier_indices = [] 
#   for col in features:
#     if col == 'type' or col == 'id':
#       continue; 
#     Q1 = np.percentile(df[col], 25) 
#     Q3 = np.percentile(df[col], 75) 
#     IQR = Q3 - Q1 

#     outlier_step =  IQR 
#     outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step)].index 
#     outlier_indices.extend(outlier_list_col) 

#   outlier_indices = Counter(outlier_indices) 
#   multiple_outliers = list(k for k, v in outlier_indices.items() if v > n) 
#   print()
#   return multiple_outliers 

# features = total.columns
# print(features)
# dup = detect_outliers(total, 2, features)

# print(len(dup))

# filter(lambda a: a < 199991, dup)
# print(len(dup))

In [6]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.preprocessing import minmax_scale
#Scaler = StandardScaler()
#Scaler = MinMaxScaler()
Scaler = RobustScaler()

total[['psfMag_u', 'psfMag_g', 'psfMag_r', 'psfMag_i', 'psfMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i','fiberMag_z', 'petroMag_u', 'petroMag_g', 'petroMag_r', 'petroMag_i','petroMag_z', 'modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i','modelMag_z']] = Scaler.fit_transform(total[['psfMag_u', 'psfMag_g', 'psfMag_r', 'psfMag_i','psfMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i','fiberMag_z', 'petroMag_u', 'petroMag_g', 'petroMag_r', 'petroMag_i','petroMag_z', 'modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i','modelMag_z']])
total.describe()
X_train = total.iloc[:64488]
test = total.iloc[64488:]
X_train.describe()


,id,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
count,64488.000000,64488.000000,64488.000000,64488.000000,64488.000000,64488.000000,64488.000000,64488.000000,64488.000000,64488.000000,64488.000000,64488.000000,64488.000000,64488.000000,64488.000000,64488.000000,64488.000000,64488.000000,64488.000000,64488.000000,64488.000000
mean,99941.905316,-0.440888,-0.610509,-0.724748,-0.463199,-0.248217,-0.245004,-0.511207,-0.200987,-0.579420,-0.550763,-0.298023,-0.379095,-0.155100,-0.503593,-0.409848,-0.173630,-0.231434,-0.625223,-0.366428,-0.590365
std,57713.351638,60.551006,78.098952,71.429795,49.204389,37.277681,38.706530,62.393275,39.493755,56.900536,62.322433,52.793235,55.677018,26.654810,66.882359,73.110655,47.078187,61.693111,70.566393,42.984477,61.730908
min,1.000000,-10112.639762,-15272.381110,-10613.103669,-10455.591191,-5678.244001,-6319.268895,-11602.894051,-7505.442572,-8019.067220,-9204.870905,-6946.168232,-7870.715868,-4433.858123,-14821.964184,-11180.473202,-8622.359085,-13633.400113,-13543.398278,-7213.844548,-9509.096047
25%,49963.250000,-0.445934,-0.487756,-0.597764,-0.626598,-0.518801,-0.451568,-0.472561,-0.580610,-0.601635,-0.502494,-0.439612,-0.487183,-0.574005,-0.583106,-0.499202,-0.404928,-0.484824,-0.576560,-0.581608,-0.491750
50%,99819.500000,-0.004827,-0.012751,-0.007763,-0.011326,-0.014129,-0.001168,-0.008512,-0.004383,-0.006172,-0.008682,0.005803,-0.002162,-0.002835,-0.003440,-0.003555,0.005029,-0.001866,-0.003929,-0.003835,-0.004194
75%,149990.500000,0.586306,0.518263,0.406081,0.393007,0.489761,0.575840,0.530319,0.420469,0.411852,0.498868,0.572441,0.499517,0.411436,0.405678,0.483754,0.617203,0.503727,0.407844,0.406629,0.488163
max,199989.000000,735.659071,23.265727,38.333118,922.841429,3846.910946,31.864033,5.285221,4423.912130,2.914047,1118.760098,956.440301,2904.089204,2598.409744,1040.839939,5843.240902,2690.202446,3900.227044,5.145284,1475.338556,15.898156


In [7]:
test.describe()

,id,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
count,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000
mean,204995.000000,0.066736,0.061072,-0.010673,0.004988,0.122063,0.056552,0.056966,-0.018017,0.007777,0.118213,0.092354,-0.026142,-0.048060,0.007070,0.121618,0.078115,0.017384,-0.056584,-0.007291,0.097471
std,2889.493756,0.688147,0.981700,0.666864,0.676733,0.667829,0.696024,0.727827,0.677906,0.647768,0.688188,1.028077,5.155568,0.736373,0.721910,0.881773,0.718180,0.722809,0.689681,0.655526,0.681238
min,199991.000000,-9.134475,-23.793021,-3.968727,-16.275280,-1.944921,-4.066601,-4.587940,-2.876028,-2.527211,-10.850148,-43.447659,-504.663990,-16.010524,-9.874967,-30.842078,-2.653775,-2.167565,-2.119094,-2.038345,-2.049339
25%,202493.000000,-0.367286,-0.399444,-0.495358,-0.439878,-0.346248,-0.378371,-0.411065,-0.511205,-0.459262,-0.370593,-0.444491,-0.527173,-0.613549,-0.569187,-0.475306,-0.404781,-0.525493,-0.617768,-0.572915,-0.473535
50%,204995.000000,0.023396,0.073237,0.053584,0.067960,0.112776,0.006291,0.049271,0.033586,0.046341,0.067234,-0.033770,0.018484,0.027120,0.031892,0.032427,-0.029036,0.019754,0.025417,0.033893,0.029920
75%,207497.000000,0.449115,0.546070,0.461263,0.481120,0.616497,0.460233,0.568644,0.480907,0.502743,0.638080,0.490879,0.550884,0.485193,0.519682,0.643290,0.489896,0.551523,0.483581,0.521104,0.660250
max,209999.000000,5.506772,62.145030,4.906334,11.156325,6.527808,7.043389,10.620511,3.854594,4.790224,3.194510,16.452865,32.262776,8.455404,12.318782,21.054859,3.986599,3.444072,3.154632,2.892797,2.352081


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, \
                                                  test_size=0.2, random_state=0)

In [9]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
clf = XGBClassifier(n_jobs=-1, random_state=0)
param_grid = {
        'silent': [False],
        'max_depth': [7],
        'learning_rate': [0.5],
        'n_estimators': [1200,1500]}

fit_params = {'eval_metric': 'mlogloss',
              'early_stopping_rounds': 10,
              'eval_set': [(X_valid, y_valid)]}

rs_clf = RandomizedSearchCV(clf, param_grid, n_iter=20,
                            n_jobs=1, verbose=2, cv=2,
                            scoring='neg_log_loss', refit=False, random_state=42)
rs_clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 12 is smaller than n_iter=20. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 2 folds for each of 12 candidates, totalling 24 fits
[CV] silent=False, n_estimators=100, max_depth=3, learning_rate=0.01 .


KeyboardInterrupt: ignored

In [0]:
%%time
from xgboost import XGBClassifier
params = {''}
XGB_clf = XGBClassifier(max_depth = 3, n_esetimators=1000, n_jobs=-1, random_state=0, learning_rate = 0.05)
XGB_clf.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)])

[0]	validation_0-merror:0.208723	validation_1-merror:0.235773
[1]	validation_0-merror:0.202345	validation_1-merror:0.229183
[2]	validation_0-merror:0.197577	validation_1-merror:0.226004
[3]	validation_0-merror:0.194224	validation_1-merror:0.225384
[4]	validation_0-merror:0.190773	validation_1-merror:0.223523
[5]	validation_0-merror:0.190095	validation_1-merror:0.223445
[6]	validation_0-merror:0.187459	validation_1-merror:0.220887
[7]	validation_0-merror:0.187013	validation_1-merror:0.219724
[8]	validation_0-merror:0.186218	validation_1-merror:0.218871
[9]	validation_0-merror:0.184745	validation_1-merror:0.218949
[10]	validation_0-merror:0.184202	validation_1-merror:0.217708
[11]	validation_0-merror:0.182535	validation_1-merror:0.216313
[12]	validation_0-merror:0.181993	validation_1-merror:0.21639
[13]	validation_0-merror:0.180791	validation_1-merror:0.216778
[14]	validation_0-merror:0.180035	validation_1-merror:0.216002
[15]	validation_0-merror:0.179647	validation_1-merror:0.215692
[16

In [0]:
XGB_pred = XGB_clf.predict_proba(test)